In [154]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from unidecode import unidecode

In [155]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# Assessing UN City Population Projections
In 2018 the UN made population projections till 2035 for around 220 major African cities. Many of these cities have since performed a nation wide census and therefore have reliable city population figures. In this notebook, we will compare the UN projections to the actual populations of cities found during any census performed after 2018.

## File structure
The UN projections data is contained in a single csv. The country specific city population data is stored in separate folders, one per country. We need to navigate through this file structure to collect the city population figures for all countries. The starting directory for this notebook contains a folder called 'all_africa_countries_pop_and_city_data'. This folder contains a folder for every single African country. Inside each African country folder (e.g. inside the 'kenya' folder) are several csv files containing data for that country. 

Below we show an example process of navigating from the starting directory to find a specific population file for kenya: 

*Starting Directory* **-->** *all_africa_countries_pop_and_city_data* **-->** *kenya* **-->** *kenya_major_cities.csv*

## Combine All City Population Data
We need to find the **major cities** csv file for each country. They use the naming convention \[country\]_major_cities.csv e.g. for kenya it is kenya_major_cities.csv. 

We will begin by listing the contents of the starting directory:

In [156]:
os.listdir()

['UN Population Prediction vs Measured Population.html',
 '.DS_Store',
 'all_africa_countries_pop_and_city_data',
 'rwanda_major_cities.csv',
 'data fixes.xlsx',
 'build_un_comparison_data.ipynb',
 'UN_city_pop_projections_long.csv',
 'UN_projections_compared',
 'compare_un_to_census.ipynb',
 'Projection Error vs Population Prediction.html',
 '.ipynb_checkpoints']

Now we will go into the **all_africa_countries_pop_and_city_data** folder:

In [157]:
os.chdir('all_africa_countries_pop_and_city_data')

From here, we again list the files in this directory: 

In [158]:
os.listdir()

['kenya',
 'Botswana',
 'Benin',
 'Tunisia',
 'Tanzania',
 'Nambia',
 'Equitorial Guinea',
 'Zambia',
 '.DS_Store',
 'Ghana',
 'Gabon',
 'Malawi',
 'libya',
 'morocco',
 'Seychelles',
 'Liberia',
 'Central africa',
 'Sudan',
 'Lesotho',
 'Mali',
 'Nigeria',
 'Dijboiti',
 'Somalia',
 'Burkina',
 'Estwatini',
 'st.helana',
 'Algeria',
 'Dr Congo',
 'Angola',
 'camaroon',
 'Senegal',
 'mayotte',
 'Eriteria',
 'Ethiopia',
 'Togo',
 'Capeverde',
 'Mazambiqui',
 'Uganda',
 'chad',
 'Niger',
 'IvoryCoast',
 'Gambia',
 'Reunion',
 'Saotome',
 'Guinea',
 'South Africa',
 'sierra leone',
 'Zimbabve',
 'madagascar',
 'Mauritania',
 'Marituis',
 'Egypt',
 'comros',
 'Burundi',
 'Rawanda',
 'Bulawayo',
 'Congo_rep',
 'guineabissau']

We can see we have a different folder for each African country. We will now create a function that takes the folder name (e.g. a country name) and will open that folder, extract a list of the files in that folder and then return to the **all_africa_countries_pop_and_city_data** folder: 

In [159]:
def get_file_names(folder_name):
    os.chdir(folder_name)
    
    file_names = os.listdir()
    
    os.chdir('..')
    return file_names

In [160]:
folders = os.listdir()
# Remove the .DS_store file that github added to folder structure
folders = [folder for folder in folders if folder[0] != '.']

files = []
for folder in folders:
    file_names = get_file_names(folder)
    files.append([folder, file_names])

In [161]:
df = pd.DataFrame(files, columns=['country', 'files'])
df.head()

,country,files
0,kenya,"[rift valley, Kenya_major_cities.csv, nyanza, ..."
1,Botswana,"[cities_botswana.csv, botswana_Agglomerations...."
2,Benin,"[Cities_population_benin.csv, Benin_major_citi..."
3,Tunisia,"[Tunisia_Age_distribution.csv, Tunisia_age_pop..."
4,Tanzania,"[Tanzania_Admin _PopbyRegionandDistricts.csv, ..."


In [162]:
def flatten_files(files): 
    flat_files = '\t'.join(files)
    flat_files = flat_files.lower()
    return flat_files.replace('_', '')


has_major_cities = [('majorcities' in flatten_files(files)) for files in df.files]
df['has_major_cities'] = has_major_cities
df

,country,files,has_major_cities
0,kenya,"[rift valley, Kenya_major_cities.csv, nyanza, ...",True
1,Botswana,"[cities_botswana.csv, botswana_Agglomerations....",False
2,Benin,"[Cities_population_benin.csv, Benin_major_citi...",True
3,Tunisia,"[Tunisia_Age_distribution.csv, Tunisia_age_pop...",True
4,Tanzania,"[Tanzania_Admin _PopbyRegionandDistricts.csv, ...",True
5,Nambia,"[Nambia_gender_population.csv, Nambia_major_ci...",True
6,Equitorial Guinea,"[EquatorialGuinea_populationon (1).csv, Equato...",True
7,Zambia,"[Zambia_population.csv, Zambia_ward_admin_popu...",True
8,Ghana,"[Ghana_population.csv, Ghana_age_population.cs...",True
9,Gabon,"[Gabon_population.csv, Gabon_cities.csv, Gabon...",True


In [163]:
df[df.has_major_cities == False]

,country,files,has_major_cities
1,Botswana,"[cities_botswana.csv, botswana_Agglomerations....",False
13,Seychelles,"[seychelles_admin_population.csv, seychelles_a...",False
19,Nigeria,"[Nigerian urban agglomerations.csv, Nigeria_Ag...",False
21,Somalia,"[Somalia_cities.csv, Somalia_population.csv]",False
24,st.helana,"[Saint Helena_places.csv, Saint Helena_populat...",False
25,Algeria,"[population_algeria.csv, Algeria_cities_popula...",False
27,Angola,"[Angola_Admin_PopbyGender.csv, Angola_Admin_Po...",False
41,Reunion,"[reunion_ arrondissements.csv, reunion_Major C...",False
44,South Africa,"[SouthAfrica_Ekurhuleni_UrbanPop.csv, SouthAfr...",False
51,comros,"[Comros_admin.csv, comros_island_population.cs...",False


In [164]:
df.has_major_cities.sum()

45

In [165]:
def collect_major_city_csv(folder, country_data):
    os.chdir(folder)
    
    files = os.listdir()
    
    for file in files:
        file_cleaned = file.lower().replace('_','')
        if 'majorcities' in file_cleaned:
            country_data[folder] = pd.read_csv(file)
    
    os.chdir('..')
    return country_data

In [166]:
country_data = {}
for folder in folders: 
    country_data = collect_major_city_csv(folder, country_data)
    
len(country_data.keys())

45

## Merge City Population Data

In [167]:
for country, data in country_data.items():
    print(country)
    print(data.columns)

kenya
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Benin
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Tunisia
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Tanzania
Index(['Unnamed: 0', 'Name', 'Adm.', 'Population Census (C) 2012-08-26'], dtype='object')
Nambia
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Equitorial Guinea
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Zambia
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Ghana
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Gabon
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Malawi
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
libya
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
morocco
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Liberia
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Central africa
Index(['Unnamed: 0', '0', '1', '2', '3'], dtype='object')
Sudan
Index(['Unnamed: 0', 'Name', 'Adm

In [168]:
def clean_df(df): 
    if df.columns[1] == '0': 
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df.columns = new_header #set the header row as the df header
    # Select all rows from 'Names' and further right 
    df = df.loc[:,'Name':]
    # Dijboiti had an extra column to right called 'Area', this removes
    if df.shape[1] > 3: 
        df = df.iloc[:,0:3]
    
    return df

In [169]:
for country, data in country_data.items():
    country_data[country] = clean_df(data)
    
for country, data in country_data.items():
    print(country)
    print(data.columns)

kenya
Index(['Name', 'Adm.', 'Population Census (C) 2019-08-24'], dtype='object', name=0)
Benin
Index(['Name', 'Adm.', 'Population Census (Cf) 2013-05-11'], dtype='object', name=0)
Tunisia
Index(['Name', 'Adm.', 'Population Estimate (E) 2022-01-01'], dtype='object', name=0)
Tanzania
Index(['Name', 'Adm.', 'Population Census (C) 2012-08-26'], dtype='object')
Nambia
Index(['Name', 'Adm.', 'Population Census (Cf) 2011-08-28'], dtype='object', name=0)
Equitorial Guinea
Index(['Name', 'Adm.', 'Population Calculation (UE-C) 2015-07-04'], dtype='object', name=0)
Zambia
Index(['Name', 'Adm.', 'Population Census (Cp) 2022-08-18'], dtype='object', name=0)
Ghana
Index(['Name', 'Adm.', 'Population Census (C) 2013-04-15'], dtype='object', name=0)
Gabon
Index(['Name', 'Adm.', 'Population Census (C) 2013-10-01'], dtype='object', name=0)
Malawi
Index(['Name', 'Adm.', 'Population Projection (P) 2020-09-03'], dtype='object', name=0)
libya
Index(['Name', 'Adm.', 'Population Estimate (E) 2020-01-01'], dty

In [170]:
for country, data in country_data.items():
    print(country)
    print(data.shape[1])

kenya
3
Benin
3
Tunisia
3
Tanzania
3
Nambia
3
Equitorial Guinea
3
Zambia
3
Ghana
3
Gabon
3
Malawi
3
libya
3
morocco
3
Liberia
3
Central africa
3
Sudan
3
Lesotho
3
Mali
3
Dijboiti
3
Burkina
3
Estwatini
3
Dr Congo
3
camaroon
3
Senegal
3
mayotte
3
Eriteria
3
Ethiopia
3
Togo
3
Capeverde
3
Mazambiqui
3
Uganda
3
chad
3
Niger
3
IvoryCoast
3
Gambia
3
Saotome
3
Guinea
3
sierra leone
3
Zimbabve
3
madagascar
3
Mauritania
3
Marituis
3
Egypt
3
Burundi
3
Congo_rep
3
guineabissau
3


In [171]:
# Go back to starting directory
os.chdir('..')
# Load rwanda data (was collected separately to the other data)
rwanda_data = pd.read_csv('rwanda_major_cities.csv')
rwanda_data

,Name,Adm.,Population Census (C) 15/08/2022
0,Kigali,KIG,"1,517,168"
1,Gisenyi,OUE,"270,000"
2,Ruhengeri,NOR,"135,000"
3,Gitarama,SUD,"85,000"
4,Butare,SUD,"75,0"


In [172]:
country_data['rwanda'] = rwanda_data

In [173]:
city_pop = pd.DataFrame()

for country, data in country_data.items():
    data['country'] = country
    city_pop = pd.concat([city_pop, data])
    
city_pop    

,Name,Adm.,Population Census (C) 2019-08-24,country,Population Census (Cf) 2013-05-11,Population Estimate (E) 2022-01-01,Population Census (C) 2012-08-26,Population Census (Cf) 2011-08-28,Population Calculation (UE-C) 2015-07-04,Population Census (Cp) 2022-08-18,Population Census (C) 2013-04-15,Population Census (C) 2013-10-01,Population Projection (P) 2020-09-03,Population Estimate (E) 2020-01-01,Population Calculation (UE-P) 2020-07-01,Population Census (Cf) 2008-03-21,Population Estimate (E) 2021-03-01,Population Census (Cp) 2008-04-22,Population Census (C) 2016-04-10,Population Census (Cf) 2009-04-01,Population Census (C) 2009-05-29,Population Census (Cf) 2019-11-16,Population Census (C) 2017-05-11,Population Estimate (E) 2004-07-01,Population Projection (P) 2015-01-01,Population Census (Cf) 2013-11-19,Population Census (C) 2017-09-05,Population Estimate (E) 2018-07-01,Population Projection (P) 2022-07-01,Population Projection (P) 2020-11-06,Population Census (Cp) 2021-06-16,Population Census (Cf) 2017-08-01,Population Projection (P) 2020-07-01,Population Census (Cf) 2009-05-20,Population Census (Cf) 2012-12-10,Population Census (C) 2021-12-14,Population Census (C) 2012-05-13,Population Census (Cf) 2014-03-01,Population Census (Cf) 2021-12-09,Population Census (Cp) 2022-04-20,Population Census (Cf) 2018-06-01,Population Census (Cf) 2013-03-25,Population Estimate (E) 2020-07-01,Population Estimate (E) 2021-07-01,Population Census (Cf) 2008-08-16,Population Projection (P) 2019-07-01,Population Census (Cfa) 2009-03-01,Population Census (C) 15/08/2022
1,Nairobi,NaN,"4,397,073",kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mombasa,CO,"1,208,333",kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nakuru,RV,"570,674",kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ruiru,CE,"490,120",kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Eldoret,RV,"475,716",kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Kigali,KIG,NaN,rwanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,517,168"
1,Gisenyi,OUE,NaN,rwanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"270,000"
2,Ruhengeri,NOR,NaN,rwanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"135,000"
3,Gitarama,SUD,NaN,rwanda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"85,000"


## Format City Population Data

In [174]:
city_pop.drop(columns = ['Adm.'], inplace = True)
city_pop = city_pop.melt(['country','Name'], var_name = 'data_source', value_name = 'population')
city_pop.dropna(inplace = True)
city_pop = city_pop.reset_index(drop='True')

In [175]:
# extract date from data_source by selecting the first 10 elements of 
# the string from the right
city_pop['date_collected'] = city_pop.data_source.str[-10:]
# Convert to datetime
city_pop['date_collected'] = pd.to_datetime(city_pop.date_collected)

city_pop.data_source = city_pop.data_source.str[:-10].str.strip()

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '15/08/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


In [176]:
city_pop

,country,Name,data_source,population,date_collected
0,kenya,Nairobi,Population Census (C),"4,397,073",2019-08-24
1,kenya,Mombasa,Population Census (C),"1,208,333",2019-08-24
2,kenya,Nakuru,Population Census (C),"570,674",2019-08-24
3,kenya,Ruiru,Population Census (C),"490,120",2019-08-24
4,kenya,Eldoret,Population Census (C),"475,716",2019-08-24
...,...,...,...,...,...
338,rwanda,Kigali,Population Census (C),"1,517,168",2022-08-15
339,rwanda,Gisenyi,Population Census (C),"270,000",2022-08-15
340,rwanda,Ruhengeri,Population Census (C),"135,000",2022-08-15
341,rwanda,Gitarama,Population Census (C),"85,000",2022-08-15


In [177]:
UN_projection_date = dt.datetime(2018, 6, 1)
city_pop_after_UN = city_pop[city_pop.date_collected >= UN_projection_date]

city_pop_after_UN = city_pop_after_UN.copy()

city_pop_after_UN.country.unique()

array(['kenya', 'Tunisia', 'Zambia', 'Malawi', 'libya', 'morocco',
       'Central africa', 'Burkina', 'Eriteria', 'Ethiopia', 'Togo',
       'Capeverde', 'Uganda', 'IvoryCoast', 'sierra leone', 'Zimbabve',
       'madagascar', 'Marituis', 'Egypt', 'Congo_rep', 'rwanda'],
      dtype=object)

In [178]:
city_pop_after_UN.data_source.unique()

array(['Population Census (C)', 'Population Estimate (E)',
       'Population Census (Cp)', 'Population Projection (P)',
       'Population Calculation (UE-P)', 'Population Census (Cf)'],
      dtype=object)

In [179]:
city_pop_after_UN['year_data_collected'] = city_pop_after_UN.date_collected.dt.year
city_pop_after_UN['month_data_collected'] = city_pop_after_UN.date_collected.dt.month
city_pop_after_UN['years_since_UN_projection'] = city_pop_after_UN.year_data_collected - 2018

In [180]:
city_pop_after_UN

,country,Name,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection
0,kenya,Nairobi,Population Census (C),"4,397,073",2019-08-24,2019,8,1
1,kenya,Mombasa,Population Census (C),"1,208,333",2019-08-24,2019,8,1
2,kenya,Nakuru,Population Census (C),"570,674",2019-08-24,2019,8,1
3,kenya,Ruiru,Population Census (C),"490,120",2019-08-24,2019,8,1
4,kenya,Eldoret,Population Census (C),"475,716",2019-08-24,2019,8,1
...,...,...,...,...,...,...,...,...
338,rwanda,Kigali,Population Census (C),"1,517,168",2022-08-15,2022,8,4
339,rwanda,Gisenyi,Population Census (C),"270,000",2022-08-15,2022,8,4
340,rwanda,Ruhengeri,Population Census (C),"135,000",2022-08-15,2022,8,4
341,rwanda,Gitarama,Population Census (C),"85,000",2022-08-15,2022,8,4


In [181]:
city_pop_after_UN['months_offset_from_midyear'] = city_pop_after_UN.month_data_collected - 6
city_pop_after_UN['year_offset_from_midyear'] = city_pop_after_UN.months_offset_from_midyear / 12
city_pop_after_UN

,country,Name,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear
0,kenya,Nairobi,Population Census (C),"4,397,073",2019-08-24,2019,8,1,2,0.166667
1,kenya,Mombasa,Population Census (C),"1,208,333",2019-08-24,2019,8,1,2,0.166667
2,kenya,Nakuru,Population Census (C),"570,674",2019-08-24,2019,8,1,2,0.166667
3,kenya,Ruiru,Population Census (C),"490,120",2019-08-24,2019,8,1,2,0.166667
4,kenya,Eldoret,Population Census (C),"475,716",2019-08-24,2019,8,1,2,0.166667
...,...,...,...,...,...,...,...,...,...,...
338,rwanda,Kigali,Population Census (C),"1,517,168",2022-08-15,2022,8,4,2,0.166667
339,rwanda,Gisenyi,Population Census (C),"270,000",2022-08-15,2022,8,4,2,0.166667
340,rwanda,Ruhengeri,Population Census (C),"135,000",2022-08-15,2022,8,4,2,0.166667
341,rwanda,Gitarama,Population Census (C),"85,000",2022-08-15,2022,8,4,2,0.166667


## Load UN Projections Data

In [182]:
# rename 'Name' column to 'city'
city_pop_after_UN.rename(columns = {'Name': 'city'}, inplace = True)
# Clean country and city columns to make lower case and remove white 
# space, helps when merging
city_pop_after_UN.city = city_pop_after_UN.city.str.lower().str.strip()
city_pop_after_UN.country = city_pop_after_UN.country.str.lower().str.strip()
city_pop_after_UN

,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear
0,kenya,nairobi,Population Census (C),"4,397,073",2019-08-24,2019,8,1,2,0.166667
1,kenya,mombasa,Population Census (C),"1,208,333",2019-08-24,2019,8,1,2,0.166667
2,kenya,nakuru,Population Census (C),"570,674",2019-08-24,2019,8,1,2,0.166667
3,kenya,ruiru,Population Census (C),"490,120",2019-08-24,2019,8,1,2,0.166667
4,kenya,eldoret,Population Census (C),"475,716",2019-08-24,2019,8,1,2,0.166667
...,...,...,...,...,...,...,...,...,...,...
338,rwanda,kigali,Population Census (C),"1,517,168",2022-08-15,2022,8,4,2,0.166667
339,rwanda,gisenyi,Population Census (C),"270,000",2022-08-15,2022,8,4,2,0.166667
340,rwanda,ruhengeri,Population Census (C),"135,000",2022-08-15,2022,8,4,2,0.166667
341,rwanda,gitarama,Population Census (C),"85,000",2022-08-15,2022,8,4,2,0.166667


In [183]:
UN_data = pd.read_csv('UN_city_pop_projections_long.csv')
UN_data

,Unnamed: 0,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population
0,0,Africa,12,Algeria,20009,Annaba,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",Urban Agglomeration,36.900000,7.766670,1950,99.741
1,1,Africa,12,Algeria,20011,Batna,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",City Proper,35.555970,6.174140,1950,13.046
2,2,Africa,12,Algeria,20015,Blida,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",Urban Agglomeration,36.480781,2.831943,1950,33.970
3,3,Africa,12,Algeria,20006,El Djazaïr (Algiers),"Censuses of 1954, 1960, 1966, 1977, 1987, 1998...",Urban Agglomeration,36.752500,3.041970,1950,516.450
4,4,Africa,12,Algeria,20020,El Djelfa,"Censuses of 1977, 1987, 1998 and 2008.",Urban Agglomeration,34.672790,3.263000,1950,10.450
...,...,...,...,...,...,...,...,...,...,...,...,...
19001,19001,Africa,894,Zambia,23277,Lusaka,"Censuses of 1950, 1963, 1969, 1980, 1990, 2000...",Urban Agglomeration,-15.413374,28.277148,2035,5182.669
19002,19002,Africa,894,Zambia,23279,Ndola,"Censuses of 1963, 1969, 1980, 1990, 2000 and 2...",City Proper,-12.958670,28.636590,2035,925.726
19003,19003,Africa,716,Zimbabwe,22510,Bulawayo,"Censuses of 1951, 1956, 1962, 1973, 1982, 1992...",City Proper,-20.150000,28.583330,2035,874.479
19004,19004,Africa,716,Zimbabwe,22511,Chitungwiza,"Censuses of 1982, 1992, 2002 and 2012.",City Proper,-18.012740,31.075550,2035,552.027


In [184]:
UN_data.Country_or_area = UN_data.Country_or_area.str.lower().str.strip()
UN_data.City = UN_data.City.str.lower().str.strip()
UN_data

,Unnamed: 0,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population
0,0,Africa,12,algeria,20009,annaba,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",Urban Agglomeration,36.900000,7.766670,1950,99.741
1,1,Africa,12,algeria,20011,batna,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",City Proper,35.555970,6.174140,1950,13.046
2,2,Africa,12,algeria,20015,blida,"Censuses of 1954, 1966, 1977, 1987, 1998 and 2...",Urban Agglomeration,36.480781,2.831943,1950,33.970
3,3,Africa,12,algeria,20006,el djazaïr (algiers),"Censuses of 1954, 1960, 1966, 1977, 1987, 1998...",Urban Agglomeration,36.752500,3.041970,1950,516.450
4,4,Africa,12,algeria,20020,el djelfa,"Censuses of 1977, 1987, 1998 and 2008.",Urban Agglomeration,34.672790,3.263000,1950,10.450
...,...,...,...,...,...,...,...,...,...,...,...,...
19001,19001,Africa,894,zambia,23277,lusaka,"Censuses of 1950, 1963, 1969, 1980, 1990, 2000...",Urban Agglomeration,-15.413374,28.277148,2035,5182.669
19002,19002,Africa,894,zambia,23279,ndola,"Censuses of 1963, 1969, 1980, 1990, 2000 and 2...",City Proper,-12.958670,28.636590,2035,925.726
19003,19003,Africa,716,zimbabwe,22510,bulawayo,"Censuses of 1951, 1956, 1962, 1973, 1982, 1992...",City Proper,-20.150000,28.583330,2035,874.479
19004,19004,Africa,716,zimbabwe,22511,chitungwiza,"Censuses of 1982, 1992, 2002 and 2012.",City Proper,-18.012740,31.075550,2035,552.027


## Find UN City Projections We Can Test

### Filter UN data by countries in Population Census Data

In [185]:
city_countries = city_pop_after_UN.country.unique()
city_countries

array(['kenya', 'tunisia', 'zambia', 'malawi', 'libya', 'morocco',
       'central africa', 'burkina', 'eriteria', 'ethiopia', 'togo',
       'capeverde', 'uganda', 'ivorycoast', 'sierra leone', 'zimbabve',
       'madagascar', 'marituis', 'egypt', 'congo_rep', 'rwanda'],
      dtype=object)

In [186]:
def see_if_matched(item, column, df):
    return len(df[df[column].str.fullmatch(item)]) > 0

for country in city_countries: 
    match = see_if_matched(country, 'Country_or_area', UN_data)
    
    print(country, match)

kenya True
tunisia True
zambia True
malawi True
libya True
morocco True
central africa False
burkina False
eriteria False
ethiopia True
togo True
capeverde False
uganda True
ivorycoast False
sierra leone True
zimbabve False
madagascar True
marituis False
egypt True
congo_rep False
rwanda True


In [187]:
city_countries = city_pop_after_UN.country.unique()
city_countries

array(['kenya', 'tunisia', 'zambia', 'malawi', 'libya', 'morocco',
       'central africa', 'burkina', 'eriteria', 'ethiopia', 'togo',
       'capeverde', 'uganda', 'ivorycoast', 'sierra leone', 'zimbabve',
       'madagascar', 'marituis', 'egypt', 'congo_rep', 'rwanda'],
      dtype=object)

In [188]:
# Improvements: 
# - set which matchers to show (not just all), maybe with list of matcher names...
# unpack the results from tupple...advanced..pronbably not worth effort 

def get_scores(df_1, df_2, key1, key2, limit=2):
    fuzzy_methods = {'Ratio':fuzz.ratio, 'Partial_ratio':fuzz.partial_ratio, 
                 'token_set_ratio':fuzz.token_set_ratio, 
                 'Token_sort_ratio':fuzz.token_sort_ratio, 
                 'partial_token_set_ratio':fuzz.partial_token_set_ratio,
                'partial_token_sort_ratio':fuzz.partial_token_sort_ratio, 
                 'WRatio':fuzz.WRatio}
    
    
    keys1 = df_1[key1].unique()
    keys2 = df_2[key2].unique()
    
    scores = [apply_fuzzy_scores(key, keys2, limit, fuzzy_methods) for key in keys1]
    
    return pd.DataFrame(scores, columns = ['key'] + list(fuzzy_methods.keys()))
    
    
def apply_fuzzy_scores(key, keys2, limit, fuzzy_methods):
    """loop through (as dic comp) every scorer for the process.extract and then 
    return list of lists of results, but also inclide key at front e.g. country """
    
    scores  = [process.extract(key, keys2, limit=limit, scorer = method)  for method in fuzzy_methods.values()]
    
    return [key] + scores

In [189]:
# match countries in city_pop_after_UN to countries in UN_data
get_scores(df_1 = city_pop_after_UN, df_2 = UN_data, key1 = 'country', 
           key2 = 'Country_or_area', limit = 2)

,key,Ratio,Partial_ratio,token_set_ratio,Token_sort_ratio,partial_token_set_ratio,partial_token_sort_ratio,WRatio
0,kenya,"[(kenya, 100), (senegal, 50)]","[(kenya, 100), (burkina faso, 60)]","[(kenya, 100), (senegal, 50)]","[(kenya, 100), (senegal, 50)]","[(kenya, 100), (burkina faso, 60)]","[(kenya, 100), (burkina faso, 60)]","[(kenya, 100), (burkina faso, 54)]"
1,tunisia,"[(tunisia, 100), (namibia, 57)]","[(tunisia, 100), (ethiopia, 57)]","[(tunisia, 100), (namibia, 57)]","[(tunisia, 100), (namibia, 57)]","[(tunisia, 100), (ethiopia, 57)]","[(tunisia, 100), (ethiopia, 57)]","[(tunisia, 100), (namibia, 57)]"
2,zambia,"[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]","[(zambia, 100), (gambia, 83)]"
3,malawi,"[(malawi, 100), (mali, 80)]","[(malawi, 100), (mali, 75)]","[(malawi, 100), (mali, 80)]","[(malawi, 100), (mali, 80)]","[(malawi, 100), (mali, 75)]","[(malawi, 100), (mali, 75)]","[(malawi, 100), (mali, 80)]"
4,libya,"[(libya, 100), (liberia, 67)]","[(libya, 100), (liberia, 60)]","[(libya, 100), (liberia, 67)]","[(libya, 100), (liberia, 67)]","[(libya, 100), (liberia, 60)]","[(libya, 100), (liberia, 60)]","[(libya, 100), (liberia, 67)]"
5,morocco,"[(morocco, 100), (cameroon, 53)]","[(morocco, 100), (cameroon, 62)]","[(morocco, 100), (cameroon, 53)]","[(morocco, 100), (cameroon, 53)]","[(morocco, 100), (cameroon, 62)]","[(morocco, 100), (cameroon, 62)]","[(morocco, 100), (cameroon, 53)]"
6,central africa,"[(central african republic, 74), (south africa...","[(central african republic, 100), (south afric...","[(central african republic, 74), (south africa...","[(central african republic, 74), (south africa...","[(central african republic, 100), (south afric...","[(central african republic, 93), (south africa...","[(central african republic, 90), (south africa..."
7,burkina,"[(burkina faso, 74), (guinea, 62)]","[(burkina faso, 100), (guinea, 67)]","[(burkina faso, 100), (guinea, 62)]","[(burkina faso, 74), (guinea, 62)]","[(burkina faso, 100), (guinea, 67)]","[(burkina faso, 100), (guinea, 67)]","[(burkina faso, 90), (guinea, 62)]"
8,eriteria,"[(eritrea, 80), (liberia, 67)]","[(eritrea, 71), (liberia, 71)]","[(eritrea, 80), (liberia, 67)]","[(eritrea, 80), (liberia, 67)]","[(eritrea, 71), (liberia, 71)]","[(eritrea, 71), (liberia, 71)]","[(eritrea, 80), (liberia, 67)]"
9,ethiopia,"[(ethiopia, 100), (tunisia, 53)]","[(ethiopia, 100), (tunisia, 57)]","[(ethiopia, 100), (tunisia, 53)]","[(ethiopia, 100), (tunisia, 53)]","[(ethiopia, 100), (tunisia, 57)]","[(ethiopia, 100), (tunisia, 57)]","[(ethiopia, 100), (tunisia, 53)]"


In [190]:
# Note - UN data does not predict Mauritius ir capeverde and so we cannot use this data.

un_country_name = {'central africa':'central african republic',
                  'burkina':'burkina faso',
                  'eriteria':'eritrea',
                  'ivorycoast':'côte d\'ivoire',
                  'zimbabve':'zimbabwe',
                  'congo_rep':'congo'}

In [191]:
# map city country name to UN name (if not mapped, will be made nan)
city_pop_after_UN['country_in_UN'] = city_pop_after_UN.country.map(un_country_name)
# replace nans with original country name
city_pop_after_UN['country_in_UN'] = city_pop_after_UN['country_in_UN'].fillna(city_pop_after_UN.country)
city_pop_after_UN.country_in_UN.unique()

array(['kenya', 'tunisia', 'zambia', 'malawi', 'libya', 'morocco',
       'central african republic', 'burkina faso', 'eritrea', 'ethiopia',
       'togo', 'capeverde', 'uganda', "côte d'ivoire", 'sierra leone',
       'zimbabwe', 'madagascar', 'marituis', 'egypt', 'congo', 'rwanda'],
      dtype=object)

### Identity Cities In UN Data That we Can test

In [192]:
cities_in_scope = pd.merge(left = UN_data, right = city_pop_after_UN, how = 'inner', 
                           left_on='Country_or_area', right_on = 'country_in_UN', 
                           suffixes =('_UN','_census'))

cities_in_scope.head()

,Unnamed: 0,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,country,city,data_source,population_census,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN
0,21,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.2979,1950,36.114,burkina,ouagadougou,Population Census (Cf),"2,415,266",2019-11-16,2019,11,1,5,0.416667,burkina faso
1,21,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.2979,1950,36.114,burkina,bobo-dioulasso,Population Census (Cf),"904,920",2019-11-16,2019,11,1,5,0.416667,burkina faso
2,21,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.2979,1950,36.114,burkina,koudougou,Population Census (Cf),"160,239",2019-11-16,2019,11,1,5,0.416667,burkina faso
3,21,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.2979,1950,36.114,burkina,saaba,Population Census (Cf),"136,011",2019-11-16,2019,11,1,5,0.416667,burkina faso
4,21,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.2979,1950,36.114,burkina,ouahigouya,Population Census (Cf),"124,587",2019-11-16,2019,11,1,5,0.416667,burkina faso


In [193]:
len(cities_in_scope.Country_or_area.unique())

19

In [194]:
cities_in_scope.Country_or_area.unique()

array(['burkina faso', 'central african republic', 'congo',
       "côte d'ivoire", 'egypt', 'eritrea', 'ethiopia', 'kenya', 'libya',
       'madagascar', 'malawi', 'morocco', 'rwanda', 'sierra leone',
       'togo', 'tunisia', 'uganda', 'zambia', 'zimbabwe'], dtype=object)

Confirmed above that all 18 matching countries in city_pop_after_UN matched in the UN data. We can now find out what cities are in the UN that we can compare to our recent census data: 

In [195]:
un_cities = cities_in_scope.City.unique()
un_cities

array(['bobo-dioulasso', 'ouagadougou', 'bangui', 'brazzaville',
       'pointe-noire', 'abidjan', 'bouake', 'al-fayyum',
       'al-iskandariyah (alexandria)', 'al-ismailiyah',
       'al-mahallah al-kubra', 'al-mansurah', 'al-qahirah (cairo)',
       'as-suways', 'aswan', 'asyut', 'az-zaqazig', "bur sa'id",
       'kafr-ad-dawwar', 'tanta', 'asmara', 'addis ababa', 'dire dawa',
       'gondar', 'mekele', 'nazret', 'eldoret', 'kisumu', 'mombasa',
       'nairobi', 'nakuru', 'ruiru', 'banghazi', 'misratah',
       'tarabulus (tripoli)', 'antananarivo', 'antsirabe', 'toamasina',
       'blantyre-limbe', 'lilongwe', 'agadir',
       'dar-el-beida (casablanca)', 'fès', 'kénitra', 'marrakech',
       'meknès', 'oujda', 'rabat', 'safi', 'tanger', 'tétouan', 'kigali',
       'freetown', 'lomé', 'safaqis', 'tunis', 'kampala', 'kitwe',
       'lusaka', 'ndola', 'bulawayo', 'chitungwiza', 'harare'],
      dtype=object)

In [196]:
len(un_cities)

63

We are trying to match with 63 UN cities.

## Match UN Data to City Population Data

Now we will compare these city names to the city names in the city population census data: 

In [197]:
unidecode('meknès')

'meknes'

Found that bracketed english names and special language characters were causing issues in matching city names, so removed both" 

In [198]:
# example string cleaning
unidecode('meknès')

'meknes'

In [199]:
# clean city names in city_pop_after_UN  
city_pop_after_UN['city_cleaned'] = [unidecode(city) 
                                     for city 
                                     in city_pop_after_UN.city]

city_pop_after_UN.city_cleaned.unique()

array(['nairobi', 'mombasa', 'nakuru', 'ruiru', 'eldoret', 'kisumu',
       'kikuyu', 'thika', 'naivasha', 'karuri', 'tunis', 'safaqis',
       'susah', 'al-qayrawan', 'banzart', 'sukrah', 'sidi husayn',
       'lusaka', 'kitwe', 'ndola', 'kabwe', 'chingola', 'lilongwe',
       'blantyre', 'mzuzu', 'zomba', 'tarabulus', 'banghazi', 'misratah',
       'casablanca', 'fes', 'tanger', 'sale', 'marrakech', 'oujda',
       'meknes', 'rabat', 'agadir', 'kenitra', 'bangui', 'bimbo',
       'begoua', 'carnot', 'berberati', 'bambari', 'bouar', 'bossangoa',
       'kaga-bandoro', 'bangassou', 'ouagadougou', 'bobo-dioulasso',
       'koudougou', 'saaba', 'ouahigouya', 'kaya', 'banfora', 'pouytenga',
       'hounde', "fada n'gourma", 'asmera', 'adis abeba', 'gonder',
       'mekele', 'adama', 'awasa', 'bahir dar', 'dire dawa', 'dese',
       'jima', 'lome', 'kara', 'sokode', 'kpalime', 'atakpame', 'dapaong',
       'tsevie', 'praia', 'mindelo', 'espargos', 'assomada', 'tarrafal',
       'porto novo

In [200]:
# match countries in city_pop_after_UN to countries in UN_data
get_scores(df_1 = cities_in_scope, df_2 = city_pop_after_UN, key1 = 'City', 
           key2 = 'city_cleaned', limit = 2)

,key,Ratio,Partial_ratio,token_set_ratio,Token_sort_ratio,partial_token_set_ratio,partial_token_sort_ratio,WRatio
0,bobo-dioulasso,"[(bobo-dioulasso, 100), (bulawayo, 45)]","[(bobo-dioulasso, 100), (bo, 100)]","[(bobo-dioulasso, 100), (adis abeba, 50)]","[(bobo-dioulasso, 100), (adis abeba, 50)]","[(bobo-dioulasso, 100), (bo, 100)]","[(bobo-dioulasso, 100), (bo, 100)]","[(bobo-dioulasso, 100), (bo, 90)]"
1,ouagadougou,"[(ouagadougou, 100), (koudougou, 80)]","[(ouagadougou, 100), (agadir, 67)]","[(ouagadougou, 100), (koudougou, 80)]","[(ouagadougou, 100), (koudougou, 80)]","[(ouagadougou, 100), (agadir, 67)]","[(ouagadougou, 100), (agadir, 67)]","[(ouagadougou, 100), (koudougou, 80)]"
2,bangui,"[(bangui, 100), (banghazi, 71)]","[(bangui, 100), (banghazi, 67)]","[(bangui, 100), (banghazi, 71)]","[(bangui, 100), (banghazi, 71)]","[(bangui, 100), (banghazi, 67)]","[(bangui, 100), (banghazi, 67)]","[(bangui, 100), (banghazi, 71)]"
3,brazzaville,"[(brazzaville, 100), (banzart, 44)]","[(brazzaville, 100), (rabat, 60)]","[(brazzaville, 100), (banzart, 44)]","[(brazzaville, 100), (banzart, 44)]","[(brazzaville, 100), (rabat, 60)]","[(brazzaville, 100), (rabat, 60)]","[(brazzaville, 100), (rabat, 54)]"
4,pointe-noire,"[(pointe-noire, 100), (porto novo, 55)]","[(pointe-noire, 100), (kitwe, 60)]","[(pointe-noire, 100), (porto novo, 55)]","[(pointe-noire, 100), (porto novo, 55)]","[(pointe-noire, 100), (kitwe, 60)]","[(pointe-noire, 100), (kitwe, 60)]","[(pointe-noire, 100), (porto novo, 55)]"
5,abidjan,"[(abidjan, 100), (rabat, 50)]","[(abidjan, 100), (casablanca, 57)]","[(abidjan, 100), (rabat, 50)]","[(abidjan, 100), (rabat, 50)]","[(abidjan, 100), (casablanca, 57)]","[(abidjan, 100), (casablanca, 57)]","[(abidjan, 100), (kaga-bandoro, 51)]"
6,bouake,"[(bouake, 100), (bouar, 73)]","[(bouake, 100), (bo, 100)]","[(bouake, 100), (bouar, 73)]","[(bouake, 100), (bouar, 73)]","[(bouake, 100), (bo, 100)]","[(bouake, 100), (bo, 100)]","[(bouake, 100), (bo, 90)]"
7,al-fayyum,"[(al-fayyum, 100), (al-qayrawan, 50)]","[(al-fayyum, 100), (shubra al-khaymah, 67)]","[(al-fayyum, 100), (al-qayrawan, 50)]","[(al-fayyum, 100), (al-qayrawan, 50)]","[(al-qayrawan, 100), (al-qahirah, 100)]","[(al-fayyum, 100), (shubra al-khaymah, 67)]","[(al-fayyum, 100), (al-iskandariyah, 86)]"
8,al-iskandariyah (alexandria),"[(al-iskandariyah, 71), (al-mahallah al-kubra,...","[(al-iskandariyah, 100), (sale, 75)]","[(al-iskandariyah, 100), (al-mansurah, 49)]","[(al-iskandariyah, 73), (al-mahallah al-kubra,...","[(al-qayrawan, 100), (al-qahirah, 100)]","[(al-iskandariyah, 93), (sale, 75)]","[(al-iskandariyah, 90), (al-qayrawan, 86)]"
9,al-ismailiyah,"[(al-iskandariyah, 71), (al-qahirah, 61)]","[(jima, 75), (al-iskandariyah, 62)]","[(al-iskandariyah, 71), (al-qahirah, 61)]","[(al-iskandariyah, 71), (al-qahirah, 61)]","[(al-qayrawan, 100), (al-qahirah, 100)]","[(jima, 75), (al-iskandariyah, 62)]","[(al-jizah, 86), (al-mahallah al-kubra, 86)]"


Identified that Token Set Ratio is best fuzzy matching method for this because it can identify either the main name or a bracketed name e.g. gets 100% match on both dar-el-beida (casablanca) when it has casablanca or dar-el-beida. 

Also gives low scores (less than 80) for incorrect matches. So generally a reliable matcher. 

In [201]:
def create_fuzzy_match_key(df_1, df_2, key1, key2, method = 'WRatio', threshold = 90):
    fuzzy_methods = {'Ratio':fuzz.ratio, 'Partial_ratio':fuzz.partial_ratio, 
                 'token_set_ratio':fuzz.token_set_ratio, 
                 'Token_sort_ratio':fuzz.token_sort_ratio, 
                 'partial_token_set_ratio':fuzz.partial_token_set_ratio,
                'partial_token_sort_ratio':fuzz.partial_token_sort_ratio, 
                 'WRatio':fuzz.WRatio}
    
    method = fuzzy_methods[method]
    
    keys1 = df_1[key1].unique()
    keys2 = df_2[key2].unique()
    
    key_map = {key:fuzzy_match(key, keys2, method, threshold) for key in keys1}
    
    df_1[key1 + '_fuzzy_match'] = df_1[key1].map(key_map)
    
    return df_1
    
    
def fuzzy_match(key, keys2, method, threshold):
    """blurg """
    
    match = process.extract(key, keys2, limit = 1, scorer = method)[0]
    score = match[1]
    string = match[0]
    
    if score >= threshold: 
        return string
    
    return np.nan

In [202]:
# drop the old matching data (used to create the cities in scope...)
cities_in_scope = cities_in_scope.loc[:,'Region':'population_UN']
# remove duplicates from match
cities_in_scope = cities_in_scope.drop_duplicates().reset_index(drop = True)

In [203]:
cities_in_scope = create_fuzzy_match_key(cities_in_scope, city_pop_after_UN, 'City', 
                       'city_cleaned', method = 'token_set_ratio', 
                       threshold = 79)
cities_in_scope

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,1950,36.114,bobo-dioulasso
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,1950,33.035,ouagadougou
2,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,1951,38.000,bobo-dioulasso
3,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,1951,37.300,ouagadougou
4,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,1952,39.536,bobo-dioulasso
...,...,...,...,...,...,...,...,...,...,...,...,...
5413,Africa,716,zimbabwe,22511,chitungwiza,"Censuses of 1982, 1992, 2002 and 2012.",City Proper,-18.01274,31.07555,2034,535.060,chitungwiza
5414,Africa,716,zimbabwe,22513,harare,"Censuses of 1951, 1956, 1969, 1982, 1992, 2002...",City Proper,-17.82935,31.05389,2034,2075.912,harare
5415,Africa,716,zimbabwe,22510,bulawayo,"Censuses of 1951, 1956, 1962, 1973, 1982, 1992...",City Proper,-20.15000,28.58333,2035,874.479,bulawayo
5416,Africa,716,zimbabwe,22511,chitungwiza,"Censuses of 1982, 1992, 2002 and 2012.",City Proper,-18.01274,31.07555,2035,552.027,chitungwiza


How many of the 63 cities ata are matching with a city from the census data? 

In [204]:
matched_cities = cities_in_scope.dropna()
len(matched_cities.City.unique())

56

Not bad, we have lost about 7 cities...presumably they did not have real wor4ld data available...

In [205]:
len(matched_cities.Country_or_area.unique())

19

We still have all 19 countries represented.

In [206]:
cities = pd.merge(left = matched_cities, right = city_pop_after_UN, 
                  left_on = ['Country_or_area','City_fuzzy_match', 'year'], 
                  right_on = ['country_in_UN', 'city_cleaned',
                              'year_data_collected'], 
                  suffixes=('_UN', '_census'))
cities.head(10)

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.177150,-4.297900,2019,924.370,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),"904,920",2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.364230,-1.538340,2019,2652.720,ouagadougou,burkina,ouagadougou,Population Census (Cf),"2,415,266",2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.361220,18.554960,2021,910.270,bangui,central africa,bangui,Population Estimate (E),"812,407",2021-03-01,2021,3,3,-3,-0.250000,central african republic,bangui
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.265830,15.283180,2019,2308.076,brazzaville,congo_rep,brazzaville,Population Projection (P),"1,982,000",2019-07-01,2019,7,1,1,0.083333,congo,brazzaville
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.776090,11.863520,2019,1175.513,pointe-noire,congo_rep,pointe-noire,Population Projection (P),"1,032,000",2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire
5,Africa,384,côte d'ivoire,21602,abidjan,"Censuses of 1950, 1955, 1975, 1988, 1998 and 2...",City Proper,5.345339,-4.026789,2021,5354.826,abidjan,ivorycoast,abidjan,Population Census (C),"5,616,633",2021-12-14,2021,12,3,6,0.500000,côte d'ivoire,abidjan
6,Africa,384,côte d'ivoire,21603,bouake,"Censuses of 1958, 1975, 1988, 1998 and 2014; E...",City Proper,7.693850,-5.030310,2021,580.586,bouake,ivorycoast,bouaké,Population Census (C),"740,000",2021-12-14,2021,12,3,6,0.500000,côte d'ivoire,bouake
7,Africa,818,egypt,22814,al-fayyum,"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,29.309950,30.841800,2021,417.077,al-fayyum,egypt,al-fayyūm,Population Estimate (E),519047.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-fayyum
8,Africa,818,egypt,22804,al-iskandariyah (alexandria),"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,31.192471,29.904596,2021,5381.101,al-iskandariyah,egypt,al-iskandariyah,Population Estimate (E),5263542.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-iskandariyah
9,Africa,818,egypt,22806,al-mahallah al-kubra,"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,30.974500,31.164990,2021,523.903,al-mahallah al-kubra,egypt,al-maḥallah al-kubrā,Population Estimate (E),592573.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-mahallah al-kubra


To do: 

- check na columns of fuzzy match...did I lose some stuff? 
Fun analysis!
- manually check the Uganda - kampala UN vs City pop De numbers because they are WAAAY off 
- Breakdown of numbers of census types
- Create comparison (+-) and calculater average and standard deviation and plot graph of different cities, maybe a cool plot conveying error and stuff...line plot with central line as correct....or big and small country size to show time since or population or something....
- Review column headers and have fun! what is interesting....


Secondly: 
- Add sections and basic text at least...hopefully...

## Fix Agglommeration Data

In [207]:
agglom_city_projs = cities[cities.City_Definition != 'City Proper'].copy()
agglom_city_projs

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.361220,18.554960,2021,910.270,bangui,central africa,bangui,Population Estimate (E),"812,407",2021-03-01,2021,3,3,-3,-0.250000,central african republic,bangui
11,Africa,818,egypt,22812,al-qahirah (cairo),"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",Metropolitan area,30.039173,31.239411,2021,21322.750,al-qahirah,egypt,al-qāhirah,Population Estimate (E),9606916.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-qahirah
28,Africa,434,libya,21781,misratah,"Censuses of 1964, 1973 and 1984; Estimate for ...",Urban Agglomeration,32.375350,15.092540,2020,881.319,misratah,libya,miṣrātah,Population Estimate (E),"360,000",2020-01-01,2020,1,2,-5,-0.416667,libya,misratah
30,Africa,450,madagascar,21792,antananarivo,"Censuses of 1975 and 1993; Estimates for 1952,...",Urban Agglomeration,-18.913680,47.536130,2018,3058.387,antananarivo,madagascar,antananarivo,Population Census (Cf),"1,274,225",2018-06-01,2018,6,0,0,0.000000,madagascar,antananarivo
35,Africa,504,morocco,21888,agadir,"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,30.424847,-9.593695,2020,923.552,agadir,morocco,agadir,Population Calculation (UE-P),"498,000",2020-07-01,2020,7,2,1,0.083333,morocco,agadir
36,Africa,504,morocco,21891,dar-el-beida (casablanca),"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,33.592780,-7.619160,2020,3752.357,casablanca,morocco,casablanca,Population Calculation (UE-P),"3,563,000",2020-07-01,2020,7,2,1,0.083333,morocco,casablanca
37,Africa,504,morocco,21893,fès,"Censuses of 1971, 1982, 1994, 2004 and 2014; E...",Urban Agglomeration,34.037150,-4.999800,2020,1224.080,fes,morocco,fès,Population Calculation (UE-P),"1,187,000",2020-07-01,2020,7,2,1,0.083333,morocco,fes
42,Africa,504,morocco,21899,rabat,"Censuses of 1952, 1960, 1971, 1982, 1994, 2004...",Urban Agglomeration,34.013250,-6.832550,2020,1884.802,rabat,morocco,rabat,Population Calculation (UE-P),"533,000",2020-07-01,2020,7,2,1,0.083333,morocco,rabat
43,Africa,504,morocco,21903,tanger,"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,35.784722,-5.812778,2020,1197.923,tanger,morocco,tanger,Population Calculation (UE-P),"1,087,000",2020-07-01,2020,7,2,1,0.083333,morocco,tanger
46,Africa,768,togo,22628,lomé,"Censuses of 1970, 1981 and 2010; Estimates for...",Urban Agglomeration,6.137480,1.212270,2020,1827.878,lome,togo,lomé,Population Projection (P),"2,173,800",2020-11-06,2020,11,2,5,0.416667,togo,lome


Logic - if it says urban agglommeration, if we have that data we switch, if we do not, we drop, we dont have appropriate comparable data...

In [208]:
url = 'https://citypopulation.de/en/world/agglomerations/'
agglom_data = pd.read_html(url)[0]
agglom_data.head()

,Rank,Name,English Name,Country,Population,Status,Remark
0,1,Guăngzhōu,Canton,China,67800000,Adm-Urb,"Northern Pearl River Delta; incl. Dōngguān, Fó..."
1,2,Tōkyō,Tokyo,Japan,40800000,Aggl,"incl. Chiba, Kawasaki, Maebashi, Sagamihara, S..."
2,3,Shànghăi,Shanghai,China,40000000,Adm-Urb,"incl. Chángshú, Chángzhōu, Sūzhōu, Wúxī"
3,4,Delhi,Delhi,India,33400000,Congl,"incl. Faridabad, Ghaziabad, Gurgaon"
4,5,Jakarta,Jakarta,Indonesia,28900000,Aggl,"incl. Bekasi, Bogor, Depok, Tangerang, Tangera..."


In [209]:
get_scores(df_1 = agglom_city_projs, df_2 = agglom_data, key1 = 'city', 
           key2 = 'Name', limit = 2)

,key,Ratio,Partial_ratio,token_set_ratio,Token_sort_ratio,partial_token_set_ratio,partial_token_sort_ratio,WRatio
0,bangui,"[(Bangui, 100), (Banjul, 67)]","[(Bangui, 100), (Palembang, 80)]","[(Bangui, 100), (Banjul, 67)]","[(Bangui, 100), (Banjul, 67)]","[(Bangui, 100), (Palembang, 80)]","[(Bangui, 100), (Palembang, 80)]","[(Bangui, 100), (Palembang, 72)]"
1,al-qāhirah,"[(Al-Qāhirah, 100), (Al-Baṣrah, 63)]","[(Al-Qāhirah, 100), (Ṣan'ā', 60)]","[(Al-Qāhirah, 100), (Al-Baṣrah, 63)]","[(Al-Qāhirah, 100), (Al-Baṣrah, 63)]","[(Al-Qāhirah, 100), (Al-Kharṭūm, 100)]","[(Al-Qāhirah, 100), (Lù'ān, 75)]","[(Al-Qāhirah, 100), (Al-Iskandariyah, 86)]"
2,miṣrātah,"[(Amritsar, 50), (Al-Baṣrah, 47)]","[(Perth, 60), (Praha, 60)]","[(Amritsar, 50), (Al-Baṣrah, 47)]","[(Amritsar, 50), (Al-Baṣrah, 47)]","[(Perth, 60), (Praha, 60)]","[(Perth, 60), (Praha, 60)]","[(Perth, 54), (Praha, 54)]"
3,antananarivo,"[(Antananarivo, 100), (San Antonio, 61)]","[(Antananarivo, 100), (Astana, 83)]","[(Antananarivo, 100), (Athínai, 56)]","[(Antananarivo, 100), (Athínai, 56)]","[(Antananarivo, 100), (Astana, 83)]","[(Antananarivo, 100), (Astana, 83)]","[(Antananarivo, 100), (Astana, 75)]"
4,agadir,"[(Agadir, 100), (Gwalior, 62)]","[(Agadir, 100), (Ouagadougou, 67)]","[(Agadir, 100), (Gwalior, 62)]","[(Agadir, 100), (Gwalior, 62)]","[(Agadir, 100), (Adis Abeba, 67)]","[(Agadir, 100), (Adis Abeba, 67)]","[(Agadir, 100), (Gwalior, 62)]"
5,casablanca,"[(Casablanca, 100), (La Habana, 63)]","[(Casablanca, 100), (Atlanta, 71)]","[(Casablanca, 100), (Astana, 62)]","[(Casablanca, 100), (Astana, 62)]","[(Casablanca, 100), (Atlanta, 71)]","[(Casablanca, 100), (Atlanta, 71)]","[(Casablanca, 100), (La Habana, 63)]"
6,fès,"[(Fès, 100), (Fresno, 44)]","[(Fès, 100), (Lagos, 33)]","[(Fès, 100), (Fresno, 50)]","[(Fès, 100), (Fresno, 50)]","[(Fès, 100), (São Paulo, 50)]","[(Fès, 100), (São Paulo, 50)]","[(Fès, 100), (Fresno, 50)]"
7,rabat,"[(Rabat, 100), (Aba, 75)]","[(Rabat, 100), (Aba, 100)]","[(Rabat, 100), (Aba, 75)]","[(Rabat, 100), (Aba, 75)]","[(Rabat, 100), (Aba, 100)]","[(Rabat, 100), (Aba, 100)]","[(Rabat, 100), (Aba, 90)]"
8,tanger,"[(Tanger, 100), (Angeles, 62)]","[(Tanger, 100), (Los Angeles, 67)]","[(Tanger, 100), (Angeles, 62)]","[(Tanger, 100), (Angeles, 62)]","[(Tanger, 100), (Dàtóng, 75)]","[(Tanger, 100), (Dàtóng, 75)]","[(Tanger, 100), (Dàtóng, 68)]"
9,lomé,"[(Lomé, 100), (Qom, 57)]","[(Lomé, 100), (Colombo, 75)]","[(Lomé, 100), (Qom, 67)]","[(Lomé, 100), (Qom, 67)]","[(Colombo, 100), (Lomé, 100)]","[(Colombo, 100), (Lomé, 100)]","[(Lomé, 100), (Colombo, 90)]"


Result - no comparable agglommeration data for Safaqis or Misratah - drop them. 

In [210]:
agglom_city_projs = create_fuzzy_match_key(agglom_city_projs, agglom_data, 'city', 
                       'Name', method = 'token_set_ratio', threshold = 100)
agglom_city_projs.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,city_fuzzy_match
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.361220,18.554960,2021,910.270,bangui,central africa,bangui,Population Estimate (E),"812,407",2021-03-01,2021,3,3,-3,-0.250000,central african republic,bangui,Bangui
11,Africa,818,egypt,22812,al-qahirah (cairo),"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",Metropolitan area,30.039173,31.239411,2021,21322.750,al-qahirah,egypt,al-qāhirah,Population Estimate (E),9606916.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-qahirah,Al-Qāhirah
28,Africa,434,libya,21781,misratah,"Censuses of 1964, 1973 and 1984; Estimate for ...",Urban Agglomeration,32.375350,15.092540,2020,881.319,misratah,libya,miṣrātah,Population Estimate (E),"360,000",2020-01-01,2020,1,2,-5,-0.416667,libya,misratah,NaN
30,Africa,450,madagascar,21792,antananarivo,"Censuses of 1975 and 1993; Estimates for 1952,...",Urban Agglomeration,-18.913680,47.536130,2018,3058.387,antananarivo,madagascar,antananarivo,Population Census (Cf),"1,274,225",2018-06-01,2018,6,0,0,0.000000,madagascar,antananarivo,Antananarivo
35,Africa,504,morocco,21888,agadir,"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,30.424847,-9.593695,2020,923.552,agadir,morocco,agadir,Population Calculation (UE-P),"498,000",2020-07-01,2020,7,2,1,0.083333,morocco,agadir,Agadir


In [211]:
agglom_numbers = pd.merge(left = agglom_city_projs, right = agglom_data, 
                  left_on = 'city_fuzzy_match', 
                  right_on = 'Name')
# convert population numbers to float
agglom_numbers.Population = agglom_numbers.Population.astype(float)
agglom_numbers['source'] = 'Population Estimate (UE)'
agglom_numbers.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,city_fuzzy_match,Rank,Name,English Name,Country,Population,Status,Remark,source
0,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.361220,18.554960,2021,910.270,bangui,central africa,bangui,Population Estimate (E),"812,407",2021-03-01,2021,3,3,-3,-0.250000,central african republic,bangui,Bangui,387,Bangui,Bangui,Central African Republic,1560000.0,Aggl,NaN,Population Estimate (UE)
1,Africa,818,egypt,22812,al-qahirah (cairo),"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",Metropolitan area,30.039173,31.239411,2021,21322.750,al-qahirah,egypt,al-qāhirah,Population Estimate (E),9606916.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-qahirah,Al-Qāhirah,12,Al-Qāhirah,Cairo,Egypt,22200000.0,Aggl,"incl. Al-Jīzah [Giza], Shubrā al-Khaymah",Population Estimate (UE)
2,Africa,450,madagascar,21792,antananarivo,"Censuses of 1975 and 1993; Estimates for 1952,...",Urban Agglomeration,-18.913680,47.536130,2018,3058.387,antananarivo,madagascar,antananarivo,Population Census (Cf),"1,274,225",2018-06-01,2018,6,0,0,0.000000,madagascar,antananarivo,Antananarivo,191,Antananarivo,Tananarive,Madagascar,2950000.0,Aggl,NaN,Population Estimate (UE)
3,Africa,504,morocco,21888,agadir,"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,30.424847,-9.593695,2020,923.552,agadir,morocco,agadir,Population Calculation (UE-P),"498,000",2020-07-01,2020,7,2,1,0.083333,morocco,agadir,Agadir,483,Agadir,Agadir,Morocco,1280000.0,Adm-Urb,NaN,Population Estimate (UE)
4,Africa,504,morocco,21891,dar-el-beida (casablanca),"Censuses of 1960, 1971, 1982, 1994, 2004 and 2...",Urban Agglomeration,33.592780,-7.619160,2020,3752.357,casablanca,morocco,casablanca,Population Calculation (UE-P),"3,563,000",2020-07-01,2020,7,2,1,0.083333,morocco,casablanca,Casablanca,118,Casablanca,Casablanca,Morocco,4700000.0,Adm-Urb,NaN,Population Estimate (UE)


Update cities data to use agglomeration estimates rather than city proper estimates. 

In [212]:
cities_to_fix = agglom_numbers.City

cities.loc[cities.City.isin(cities_to_fix),
           ['population', 'data_source']] = (
    agglom_numbers[['Population', 'source']].values
)

Drop 'misratah' and 'safaqis' from dataset (no corresponding census data)

In [213]:
cities_to_drop = ['misratah', 'safaqis']
rows_to_drop = cities[cities.City.isin(cities_to_drop)].index

cities.drop(rows_to_drop, inplace = True)
cities = cities.reset_index(drop = True)

Fix years since data collected....



In [214]:
columns_to_fix = ['date_collected', 'year_data_collected',
       'month_data_collected', 'years_since_UN_projection',
       'months_offset_from_midyear', 'year_offset_from_midyear']

cities.loc[cities.City.isin(cities_to_fix),columns_to_fix]

,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear
2,2021-03-01,2021,3,3,-3,-0.250000
11,2021-07-01,2021,7,3,1,0.083333
29,2018-06-01,2018,6,0,0,0.000000
34,2020-07-01,2020,7,2,1,0.083333
35,2020-07-01,2020,7,2,1,0.083333
36,2020-07-01,2020,7,2,1,0.083333
41,2020-07-01,2020,7,2,1,0.083333
42,2020-07-01,2020,7,2,1,0.083333
45,2020-11-06,2020,11,2,5,0.416667
46,2022-01-01,2022,1,4,-5,-0.416667


Agglom data estimated for 1st jan 2023...

In [215]:
new_cols = [dt.datetime(2023, 1,1), 2023, 1, 2023-2018, 1-6, (1-6)/12]
new_cols

[datetime.datetime(2023, 1, 1, 0, 0), 2023, 1, 5, -5, -0.4166666666666667]

In [216]:
cities.loc[cities.City.isin(cities_to_fix),columns_to_fix] = new_cols

In [217]:
cities.loc[cities.City.isin(cities_to_fix),columns_to_fix]

,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear
2,2023-01-01,2023,1,5,-5,-0.416667
11,2023-01-01,2023,1,5,-5,-0.416667
29,2023-01-01,2023,1,5,-5,-0.416667
34,2023-01-01,2023,1,5,-5,-0.416667
35,2023-01-01,2023,1,5,-5,-0.416667
36,2023-01-01,2023,1,5,-5,-0.416667
41,2023-01-01,2023,1,5,-5,-0.416667
42,2023-01-01,2023,1,5,-5,-0.416667
45,2023-01-01,2023,1,5,-5,-0.416667
46,2023-01-01,2023,1,5,-5,-0.416667


Update UN population estimate to the 2023 estimate (or half way between 2023 and 2022...thqts another improvement...) 

In [218]:
UN_data_2023 = UN_data[(UN_data.City.isin(cities_to_fix)) 
                       & (UN_data.year == 2023)]

for city, pop in zip(UN_data_2023.City, UN_data_2023.population):
    cities.loc[cities.City == city, ['population_UN', 'year']] = [pop, 2023]

In [219]:
cities

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.177150,-4.297900,2019,924.370,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),"904,920",2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.364230,-1.538340,2019,2652.720,ouagadougou,burkina,ouagadougou,Population Census (Cf),"2,415,266",2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.361220,18.554960,2023,958.335,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.265830,15.283180,2019,2308.076,brazzaville,congo_rep,brazzaville,Population Projection (P),"1,982,000",2019-07-01,2019,7,1,1,0.083333,congo,brazzaville
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.776090,11.863520,2019,1175.513,pointe-noire,congo_rep,pointe-noire,Population Projection (P),"1,032,000",2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire
5,Africa,384,côte d'ivoire,21602,abidjan,"Censuses of 1950, 1955, 1975, 1988, 1998 and 2...",City Proper,5.345339,-4.026789,2021,5354.826,abidjan,ivorycoast,abidjan,Population Census (C),"5,616,633",2021-12-14,2021,12,3,6,0.500000,côte d'ivoire,abidjan
6,Africa,384,côte d'ivoire,21603,bouake,"Censuses of 1958, 1975, 1988, 1998 and 2014; E...",City Proper,7.693850,-5.030310,2021,580.586,bouake,ivorycoast,bouaké,Population Census (C),"740,000",2021-12-14,2021,12,3,6,0.500000,côte d'ivoire,bouake
7,Africa,818,egypt,22814,al-fayyum,"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,29.309950,30.841800,2021,417.077,al-fayyum,egypt,al-fayyūm,Population Estimate (E),519047.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-fayyum
8,Africa,818,egypt,22804,al-iskandariyah (alexandria),"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,31.192471,29.904596,2021,5381.101,al-iskandariyah,egypt,al-iskandariyah,Population Estimate (E),5263542.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-iskandariyah
9,Africa,818,egypt,22806,al-mahallah al-kubra,"Censuses of 1960, 1966, 1976, 1986, 1996 and 2...",City Proper,30.974500,31.164990,2021,523.903,al-mahallah al-kubra,egypt,al-maḥallah al-kubrā,Population Estimate (E),592573.0,2021-07-01,2021,7,3,1,0.083333,egypt,al-mahallah al-kubra


## Format City Populations Data

In [220]:
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Region                      54 non-null     object        
 1   Country_Code                54 non-null     int64         
 2   Country_or_area             54 non-null     object        
 3   City_Code                   54 non-null     int64         
 4   City                        54 non-null     object        
 5   data_sources_UN             54 non-null     object        
 6   City_Definition             54 non-null     object        
 7   Latitude                    54 non-null     float64       
 8   Longitude                   54 non-null     float64       
 9   year                        54 non-null     int64         
 10  population_UN               54 non-null     float64       
 11  City_fuzzy_match            54 non-null     object        
 

In [221]:
# convert UN populations from units of thousands to the 
# full number e.g. 100 thousand becomes 100000
cities.population_UN = cities.population_UN * 1000
# Cast all city census population figures as strings (some are floats)
cities.population = cities.population.astype(str)
# remove ',' seperators from poulation numbers and cast asfloats
cities.population = cities.population.str.replace(',','').astype(float)

In [222]:
cities.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,2019,924370.0,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),904920.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,2019,2652720.0,ouagadougou,burkina,ouagadougou,Population Census (Cf),2415266.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.36122,18.55496,2023,958335.0,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.26583,15.28318,2019,2308076.0,brazzaville,congo_rep,brazzaville,Population Projection (P),1982000.0,2019-07-01,2019,7,1,1,0.083333,congo,brazzaville
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.77609,11.86352,2019,1175513.0,pointe-noire,congo_rep,pointe-noire,Population Projection (P),1032000.0,2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire


In [223]:
# Calculate difference between UN estimate and city population census. 
# The smaller the magnitude, the smaller the error. Positive means 
# over-estimate, negative means under estimate. 
cities['UN_proj_error'] = cities.population_UN - cities.population
# Calculate error as percentage of the population projections (allows us
# to gauge how much error to expect as a percentage in a given UN estimate. 
cities['UN_proj_error_%'] = cities.UN_proj_error / cities.population_UN
cities['mag_UN_proj_error'] = cities.UN_proj_error.abs()
cities['mag_UN_proj_error_%'] = cities['UN_proj_error_%'].abs()

In [224]:
cities['source_split'] = cities.data_sources_UN.str.split(';')
cities['latest_census_data_UN'] = cities.source_split.str[0]
cities['latest_estimate_data_UN'] = cities.source_split.str[1]
# remove period '.' at end of some estimate data (causes issues extracting dates from string)
cities.latest_estimate_data_UN = cities.latest_estimate_data_UN.str.replace('.','', regex = False)
cities.drop(columns = ['source_split'], inplace = True)
cities.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,UN_proj_error,UN_proj_error_%,mag_UN_proj_error,mag_UN_proj_error_%,latest_census_data_UN,latest_estimate_data_UN
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,2019,924370.0,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),904920.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso,19450.0,0.021041,19450.0,0.021041,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2006",Estimate for 2015
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,2019,2652720.0,ouagadougou,burkina,ouagadougou,Population Census (Cf),2415266.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou,237454.0,0.089513,237454.0,0.089513,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2006",Estimate for 2015
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.36122,18.55496,2023,958335.0,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui,-601665.0,-0.627823,601665.0,0.627823,"Censuses of 1955, 1964, 1975, 1988 and 2003",Estimate for 1966
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.26583,15.28318,2019,2308076.0,brazzaville,congo_rep,brazzaville,Population Projection (P),1982000.0,2019-07-01,2019,7,1,1,0.083333,congo,brazzaville,326076.0,0.141276,326076.0,0.141276,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2007",Estimate for 1950
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.77609,11.86352,2019,1175513.0,pointe-noire,congo_rep,pointe-noire,Population Projection (P),1032000.0,2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire,143513.0,0.122085,143513.0,0.122085,"Censuses of 1958, 1974, 1984, 1996 and 2007",Sample Survey of 1962


In [225]:
def extract_dates(string):
    if string != string:
        return np.nan
    return [int(item) for item in string.split(' ') if item.isdigit()]

cities.latest_census_data_UN = [extract_dates(string) for string in cities.latest_census_data_UN]
cities.latest_estimate_data_UN = [extract_dates(string) for string in cities.latest_estimate_data_UN]

cities.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,UN_proj_error,UN_proj_error_%,mag_UN_proj_error,mag_UN_proj_error_%,latest_census_data_UN,latest_estimate_data_UN
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,2019,924370.0,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),904920.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso,19450.0,0.021041,19450.0,0.021041,"[1996, 2006]",[2015]
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,2019,2652720.0,ouagadougou,burkina,ouagadougou,Population Census (Cf),2415266.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou,237454.0,0.089513,237454.0,0.089513,"[1996, 2006]",[2015]
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.36122,18.55496,2023,958335.0,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui,-601665.0,-0.627823,601665.0,0.627823,"[1988, 2003]",[1966]
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.26583,15.28318,2019,2308076.0,brazzaville,congo_rep,brazzaville,Population Projection (P),1982000.0,2019-07-01,2019,7,1,1,0.083333,congo,brazzaville,326076.0,0.141276,326076.0,0.141276,"[1996, 2007]",[1950]
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.77609,11.86352,2019,1175513.0,pointe-noire,congo_rep,pointe-noire,Population Projection (P),1032000.0,2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire,143513.0,0.122085,143513.0,0.122085,"[1996, 2007]",[1962]


In [226]:
def get_max(item):
    if item != item:
        # set as 0 to allow comparisons (nan values return false for all conditions)
        return 0
    return max(item)

cities.latest_census_data_UN = [get_max(dates) for dates in cities.latest_census_data_UN]
cities.latest_estimate_data_UN = [get_max(dates) for dates in cities.latest_estimate_data_UN]
cities.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,UN_proj_error,UN_proj_error_%,mag_UN_proj_error,mag_UN_proj_error_%,latest_census_data_UN,latest_estimate_data_UN
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,2019,924370.0,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),904920.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso,19450.0,0.021041,19450.0,0.021041,2006,2015
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,2019,2652720.0,ouagadougou,burkina,ouagadougou,Population Census (Cf),2415266.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou,237454.0,0.089513,237454.0,0.089513,2006,2015
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.36122,18.55496,2023,958335.0,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui,-601665.0,-0.627823,601665.0,0.627823,2003,1966
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.26583,15.28318,2019,2308076.0,brazzaville,congo_rep,brazzaville,Population Projection (P),1982000.0,2019-07-01,2019,7,1,1,0.083333,congo,brazzaville,326076.0,0.141276,326076.0,0.141276,2007,1950
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.77609,11.86352,2019,1175513.0,pointe-noire,congo_rep,pointe-noire,Population Projection (P),1032000.0,2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire,143513.0,0.122085,143513.0,0.122085,2007,1962


In [227]:
cities['most_recent_UN_source'] = np.where(cities.latest_census_data_UN 
                                           > cities.latest_estimate_data_UN, 
                                           'Census', 'Estimate')

cities['years_since_latest_UN_source'] = np.where(cities.latest_census_data_UN 
                                           > cities.latest_estimate_data_UN, 
                                                  2018 - cities.latest_census_data_UN, 
                                                  2018 - cities.latest_estimate_data_UN)

cities.head()

,Region,Country_Code,Country_or_area,City_Code,City,data_sources_UN,City_Definition,Latitude,Longitude,year,population_UN,City_fuzzy_match,country,city,data_source,population,date_collected,year_data_collected,month_data_collected,years_since_UN_projection,months_offset_from_midyear,year_offset_from_midyear,country_in_UN,city_cleaned,UN_proj_error,UN_proj_error_%,mag_UN_proj_error,mag_UN_proj_error_%,latest_census_data_UN,latest_estimate_data_UN,most_recent_UN_source,years_since_latest_UN_source
0,Africa,854,burkina faso,23191,bobo-dioulasso,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,11.17715,-4.29790,2019,924370.0,bobo-dioulasso,burkina,bobo-dioulasso,Population Census (Cf),904920.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,bobo-dioulasso,19450.0,0.021041,19450.0,0.021041,2006,2015,Estimate,3
1,Africa,854,burkina faso,23192,ouagadougou,"Censuses of 1951, 1960, 1975, 1985, 1996 and 2...",City Proper,12.36423,-1.53834,2019,2652720.0,ouagadougou,burkina,ouagadougou,Population Census (Cf),2415266.0,2019-11-16,2019,11,1,5,0.416667,burkina faso,ouagadougou,237454.0,0.089513,237454.0,0.089513,2006,2015,Estimate,3
2,Africa,140,central african republic,20410,bangui,"Censuses of 1955, 1964, 1975, 1988 and 2003; E...",Urban Agglomeration,4.36122,18.55496,2023,958335.0,bangui,central africa,bangui,Population Estimate (UE),1560000.0,2023-01-01,2023,1,5,-5,-0.416667,central african republic,bangui,-601665.0,-0.627823,601665.0,0.627823,2003,1966,Census,15
3,Africa,178,congo,20848,brazzaville,"Censuses of 1955, 1961, 1974, 1984, 1996 and 2...",City Proper,-4.26583,15.28318,2019,2308076.0,brazzaville,congo_rep,brazzaville,Population Projection (P),1982000.0,2019-07-01,2019,7,1,1,0.083333,congo,brazzaville,326076.0,0.141276,326076.0,0.141276,2007,1950,Census,11
4,Africa,178,congo,20849,pointe-noire,"Censuses of 1958, 1974, 1984, 1996 and 2007; S...",City Proper,-4.77609,11.86352,2019,1175513.0,pointe-noire,congo_rep,pointe-noire,Population Projection (P),1032000.0,2019-07-01,2019,7,1,1,0.083333,congo,pointe-noire,143513.0,0.122085,143513.0,0.122085,2007,1962,Census,11


In [228]:
# Extract high level category of data source from data source 
# name e.g. a Population Census (Cf) becomes just Population Census. 
cities['data_source_type'] = cities.data_source.str.split('(').str[0].str.strip()

# has 3 types, population census, population estimate (based on birth rates, 
# death rates and mortality), population projections (often poor) and population 
# calculations. In this case, all calculations were based on projections, so we
# will merge them together into the same category. 
cities.data_source_type = cities.data_source_type.str.replace('Calculation',
                                                              'Projection')

In [230]:
cities.to_csv('UN_projections_compared.csv', index = False)